In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import datetime
import gc

## Train dataset

In [2]:
DATA_PATH = './data/'

In [3]:
train_df = pd.read_csv(DATA_PATH + 'train_cleaned.csv')

In [4]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
0,105,0,2016-01-01 00:00:00,23.3036,1,Education,50623,NaN,5.0
1,106,0,2016-01-01 00:00:00,0.3746,1,Education,5374,NaN,4.0
2,106,3,2016-01-01 00:00:00,0.0000,1,Education,5374,NaN,4.0
3,107,0,2016-01-01 00:00:00,175.1840,1,Education,97532,2005.0,10.0
4,108,0,2016-01-01 00:00:00,91.2653,1,Education,81580,1913.0,5.0


In [5]:
def building_preprocess(df):
    df.drop(["year_built", "floor_count"], axis=1, inplace=True)
    le = LabelEncoder()
    df.primary_use = le.fit_transform(df.primary_use)

    del le
    gc.collect()

In [6]:
building_preprocess(train_df)

In [7]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet
0,105,0,2016-01-01 00:00:00,23.3036,1,0,50623
1,106,0,2016-01-01 00:00:00,0.3746,1,0,5374
2,106,3,2016-01-01 00:00:00,0.0000,1,0,5374
3,107,0,2016-01-01 00:00:00,175.1840,1,0,97532
4,108,0,2016-01-01 00:00:00,91.2653,1,0,81580


### weather metadata

In [8]:
weather_df = pd.read_csv(DATA_PATH + 'weather_train.csv')

In [9]:
weather_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [10]:
def weather_preprocess(df):
    df.drop(["sea_level_pressure", "wind_direction", "wind_speed"], axis=1, inplace=True)
    df = df.groupby("site_id").apply(lambda group: group.interpolate(limit_direction="both"))
    return df

In [11]:
weather_df = weather_preprocess(weather_df)

In [12]:
weather_df.isnull().sum()

site_id                  0
timestamp                0
air_temperature          0
cloud_coverage       17228
dew_temperature          0
precip_depth_1_hr    26273
dtype: int64

In [13]:
# 1, 5, 12 precip_depth_1_hr
# site 7, 11 cloud_coverage
site_ids = weather_df['site_id'].unique()
for site_id in site_ids:
    null_sum = weather_df.query('site_id == @site_id')['cloud_coverage'].isnull().sum()
    if null_sum != 0:
        rows = weather_df.query('site_id == @site_id').shape[0]
        print('site_id {}, rows {}, rows of null {}'.format(site_id, rows, null_sum))

site_id 7, rows 8614, rows of null 8614
site_id 11, rows 8614, rows of null 8614


### Merge  datasets

In [14]:
train_df = train_df.merge(weather_df, on=['site_id','timestamp'], how='left')

In [15]:
train_df.timestamp = pd.to_datetime(train_df.timestamp, format='%Y-%m-%d %H:%M:%S')

In [16]:
train_df["meter_reading"] = train_df["meter_reading"].map(np.log1p)

In [17]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr
0,105,0,2016-01-01,3.190624,1,0,50623,3.8,0.0,2.4,NaN
1,106,0,2016-01-01,0.318163,1,0,5374,3.8,0.0,2.4,NaN
2,106,3,2016-01-01,0.000000,1,0,5374,3.8,0.0,2.4,NaN
3,107,0,2016-01-01,5.171529,1,0,97532,3.8,0.0,2.4,NaN
4,108,0,2016-01-01,4.524668,1,0,81580,3.8,0.0,2.4,NaN


In [18]:
del weather_df
gc.collect()

60

### Creating time-based features

In [19]:
def add_timebase_features(df):
    df["hour"] = df["timestamp"].dt.hour
    df["weekend"] = df["timestamp"].dt.weekday
    df['square_feet'] =  np.log1p(df['square_feet'])
    return df

In [20]:
train_df = add_timebase_features(train_df)

In [21]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,hour,weekend
0,105,0,2016-01-01,3.190624,1,0,10.832181,3.8,0.0,2.4,NaN,0,4
1,106,0,2016-01-01,0.318163,1,0,8.589514,3.8,0.0,2.4,NaN,0,4
2,106,3,2016-01-01,0.000000,1,0,8.589514,3.8,0.0,2.4,NaN,0,4
3,107,0,2016-01-01,5.171529,1,0,11.487946,3.8,0.0,2.4,NaN,0,4
4,108,0,2016-01-01,4.524668,1,0,11.309352,3.8,0.0,2.4,NaN,0,4


In [22]:
train_df.tail()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,hour,weekend
19455577,1444,0,2016-12-31 23:00:00,2.277267,15,1,9.884305,1.7,2.0,-5.6,-1.0,23,5
19455578,1445,0,2016-12-31 23:00:00,1.762159,15,0,8.366138,1.7,2.0,-5.6,-1.0,23,5
19455579,1446,0,2016-12-31 23:00:00,0.000000,15,1,9.329545,1.7,2.0,-5.6,-1.0,23,5
19455580,1447,0,2016-12-31 23:00:00,5.078761,15,4,10.301458,1.7,2.0,-5.6,-1.0,23,5
19455581,1448,0,2016-12-31 23:00:00,1.348073,15,6,11.432496,1.7,2.0,-5.6,-1.0,23,5


In [23]:
train_df.drop(['timestamp', 'building_id', 'site_id'], axis=1, inplace=True)
gc.collect()

20

In [24]:
# Training = 70% of the data
# Validation = 30% of the data
# Randomize the datset
np.random.seed(5)
l = list(train_df.index)
np.random.shuffle(l)
train_df = train_df.loc[l]

In [25]:
rows = train_df.shape[0]
train = int(.7 * rows)

In [26]:
train_df.head()

,meter,meter_reading,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,hour,weekend
8596043,0,5.805788,9,12.903595,20.0,2.947368,18.3,-1.0,11,4
1178738,0,1.916923,0,9.584865,13.9,4.000000,9.4,0.0,23,5
19010666,0,3.526361,6,10.671162,0.0,4.000000,-1.1,3.0,1,5
9186732,1,8.203539,0,12.456835,23.9,0.000000,20.0,0.0,3,1
13400078,0,2.370244,0,10.096254,15.0,0.000000,15.0,NaN,6,1


In [27]:
columns = ['meter_reading', 'meter', 'primary_use', 'square_feet',
           'air_temperature', 'cloud_coverage', 'dew_temperature',
           'precip_depth_1_hr', 'hour', 'weekend']

In [28]:
train_df.iloc[:train].to_csv(DATA_PATH + 'ashrae_train.csv',
                             index=False,
                             header=False,
                             columns=columns)

In [29]:
train_df.iloc[train:].to_csv(DATA_PATH + 'ashrae_validation.csv',
                             index=False,
                             header=False,
                             columns=columns)

## Test dataset

In [30]:
test_df = pd.read_csv(DATA_PATH + 'test.csv')

In [31]:
building_df = pd.read_csv(DATA_PATH + 'building_metadata.csv')
test_df = test_df.merge(building_df, on='building_id', how='left')

In [32]:
building_preprocess(test_df)

In [33]:
test_df.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet
0,0,0,0,2017-01-01 00:00:00,0,0,7432
1,1,1,0,2017-01-01 00:00:00,0,0,2720
2,2,2,0,2017-01-01 00:00:00,0,0,5376
3,3,3,0,2017-01-01 00:00:00,0,0,23685
4,4,4,0,2017-01-01 00:00:00,0,0,116607


In [34]:
weather_test_df = pd.read_csv(DATA_PATH + 'weather_test.csv')
weather_test_df = weather_preprocess(weather_test_df)

In [35]:
test_df = test_df.merge(weather_test_df, on=['site_id','timestamp'], how='left')
test_df.timestamp = pd.to_datetime(test_df.timestamp, format='%Y-%m-%d %H:%M:%S')

In [36]:
del weather_test_df
del building_df
gc.collect()

40

In [37]:
test_df = add_timebase_features(test_df)
test_df.drop(['timestamp', 'building_id', 'site_id'], axis=1, inplace=True)

In [38]:
test_df.head()

,row_id,meter,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,hour,weekend
0,0,0,0,8.913685,17.8,4.0,11.7,0.0,0,6
1,1,0,0,7.908755,17.8,4.0,11.7,0.0,0,6
2,2,0,0,8.589886,17.8,4.0,11.7,0.0,0,6
3,3,0,0,10.072639,17.8,4.0,11.7,0.0,0,6
4,4,0,0,11.666573,17.8,4.0,11.7,0.0,0,6


In [39]:
test_df.to_csv(DATA_PATH + 'ashrae_test.csv', index=False)

In [40]:
del train_df, test_df
gc.collect()

109

In [41]:
# Write Column List
with open(DATA_PATH + 'ashrae_train_column_list.txt', 'w') as f:
    f.write(','.join(columns))